# Order and Delivery to Google Earth Engine

In this notebook we're going to cover how to order data using the [Orders API](https://developers.planet.com/apis/orders/) and deliver it to [Google Earth Engine (GEE)](https://earthengine.google.com/) using [Planet’s GEE Delivery integration](https://developers.planet.com/docs/integrations/gee/).

This example demonstrates how to:
1. Define the data to be ordered
2. Build a cloud delivery configuration object (`delivery_config`), which tells the Orders API where to deliver the data
3. Build an order request to be sent off to the Orders API, `iowa_order`
4. Create the order and have it deliver to your GEE project

**Prerequsites:** A known AOI, Scene IDs, GEE project with EE API enabled containing a pre-existing GEE [ImageCollection](https://developers.google.com/earth-engine/guides/ic_creating), and your account must have a download quota. 

For this example, here are the aformentioned prerequisets as they are defined in this notebook:
- AOI: `iowa_aoi`
- Scene IDs: `iowa_images`
- GEE project: `planet-devrel-dev`
- GEE ImageCollection: `gee-integration-testing`

In [1]:
import planet

## Define data to be ordered

In [2]:
# The area of interest (AOI) defined as a polygon
iowa_aoi = {
    "type":
    "Polygon",
    "coordinates": [[[-91.198465, 42.893071], [-91.121931, 42.893071],
                     [-91.121931, 42.946205], [-91.198465, 42.946205],
                     [-91.198465, 42.893071]]]
}

# The scene IDs we wish to order
iowa_images = ['20200925_161029_69_2223', '20200925_161027_48_2223']

## Define cloud delivery location
This is the *key concept* of this notebook. Here, we are defining a cloud delivery configuration object, `cloud_config`, where we are defining the destination to be Google Earth Engine, in the project named `planet-devrel-dev`, in the ImageCollection named `gee-integration-testing`.

In [3]:
# Google Earth Engine configuration
cloud_config = planet.order_request.google_earth_engine(
    project='planet-devrel-dev', collection='gee-integration-testing')
# Order delivery configuration
delivery_config = planet.order_request.delivery(cloud_config=cloud_config)

print(delivery_config)

{'google_earth_engine': {'project': 'planet-devrel-dev', 'collection': 'gee-integration-testing'}}


## Build order request
Here we are building an order request, where we specify the products we wish to order. Spesifically, here we are requesting the IDs defined in `iowa_images`, as `analytic_udm2` assets, from the Planet Scope Scene product, `PSScene`.

In [4]:
# Product description for the order request
data_products = [
    planet.order_request.product(item_ids=iowa_images,
                                 product_bundle='analytic_udm2',
                                 item_type='PSScene')
]
# Clip the images to the AOI's perimeter
tools = [planet.order_request.clip_tool(iowa_aoi)]

# Build the order request
iowa_order = planet.order_request.build_request(name='iowa_order',
                                                products=data_products,
                                                delivery=delivery_config,
                                                tools=tools)

print(iowa_order)

{'name': 'iowa_order', 'products': [{'item_ids': ['20200925_161029_69_2223', '20200925_161027_48_2223'], 'item_type': 'PSScene', 'product_bundle': 'analytic_udm2'}], 'delivery': {'google_earth_engine': {'project': 'planet-devrel-dev', 'collection': 'gee-integration-testing'}}, 'tools': [{'clip': {'aoi': {'type': 'Polygon', 'coordinates': [[[-91.198465, 42.893071], [-91.121931, 42.893071], [-91.121931, 42.946205], [-91.198465, 42.946205], [-91.198465, 42.893071]]]}}}]}


## Create and deliver the order
Define a function that will create an order and update you with its progress. In this case, since we specified the delivery destination to be Google Earth Engine with `delivery_config`, it will also wait for it to be delivered to your project.

In [5]:
async def create_and_deliver_order(order_request, client):
    '''Create and deliver an order.

    Parameters:
        order_request: An order request
        client: An Order client object
    '''
    with planet.reporting.StateBar(state='creating') as reporter:
        # Place an order to the Orders API
        order = await client.create_order(order_request)
        reporter.update(state='created', order_id=order['id'])
        # Wait while the order is being completed
        await client.wait(order['id'], callback=reporter.update_state)

    # Grab the details of the orders
    order_details = await client.get_order(order_id=order['id'])

    return order_details

Create and deliver the order `iowa_order` to the GEE project `planet-devrel-dev` in the ImageCollection `gee-integration-testing`.

In [6]:
async with planet.Session() as ps:
    # The Orders API client
    client = planet.OrdersClient(ps)
    # Create the order and deliver it to GEE
    order_details = await create_and_deliver_order(iowa_order, client)

04:51 - order c2e1cecd-1c8b-4f6f-b29b-2e44a813e2fb - state: success


Show the result of the order.

In [14]:
order_results = order_details['_links']['results']
for i in range(len(order_results)):
    print(order_results[i])

{'delivery': 'success', 'expires_at': '2022-11-15T23:11:57.623Z', 'location': 'https://api.planet.com/compute/ops/download/?token=eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJleHAiOjE2Njg1NTM5MTcsInN1YiI6ImtMcXdiVEpKVFZieXdsSnNrZDF6Y083YVd2aW9pZE9zaTIzL0VkaTNMNUhjT1pGcndXVCtnYUt4MGxXOVpxbUIzYVJxTTNXOHFuUGRTU2djR1lpcUZ3PT0iLCJ0b2tlbl90eXBlIjoiZG93bmxvYWQtYXNzZXQtc3RhY2siLCJhb2kiOiIiLCJhc3NldHMiOlt7Iml0ZW1fdHlwZSI6IiIsImFzc2V0X3R5cGUiOiIiLCJpdGVtX2lkIjoiIn1dLCJ1cmwiOiJodHRwczovL3N0b3JhZ2UuZ29vZ2xlYXBpcy5jb20vY29tcHV0ZS1vcmRlcnMtbGl2ZS9jMmUxY2VjZC0xYzhiLTRmNmYtYjI5Yi0yZTQ0YTgxM2UyZmIvUFNTY2VuZS8yMDIwMDkyNV8xNjEwMjdfNDhfMjIyM18zQl9BbmFseXRpY01TX21ldGFkYXRhX2NsaXAueG1sP1gtR29vZy1BbGdvcml0aG09R09PRzQtUlNBLVNIQTI1Nlx1MDAyNlgtR29vZy1DcmVkZW50aWFsPWNvbXB1dGUtZ2NzLXN2Y2FjYyU0MHBsYW5ldC1jb21wdXRlLXByb2QuaWFtLmdzZXJ2aWNlYWNjb3VudC5jb20lMkYyMDIyMTExNCUyRmF1dG8lMkZzdG9yYWdlJTJGZ29vZzRfcmVxdWVzdFx1MDAyNlgtR29vZy1EYXRlPTIwMjIxMTE0VDIzMTE1N1pcdTAwMjZYLUdvb2ctRXhwaXJlcz04NjM5OVx1MDAyNlgtR29vZy1TaWduYXR1cmU9N